# This notebook is being prepared for final project in data science professional certification course by IBM.

In [2]:
import numpy
import pandas
print("Hello capstone project course.")

Hello capstone project course.
